In [1]:
import re

import pandas as pd
import scipy.sparse as sparse
import numpy as np

import scipy.sparse as sparse
import cPickle as pickle

## Interacting with the dataframe

In [9]:
# modify this accordingly to load the full dataframe
# e.g.
# df_fpath = "/vagrant/xdata/uspto_grant_data_icpr_section_H.h5"
# df = pd.read_hdf(df_fpath)

pickle_fpath = "uspto_grants_all_H_sections.hdf5"
full_df  = pd.read_hdf(pickle_fpath)
full_df.reset_index(inplace=True,drop=True)

In [29]:
#selector = [True if re.match("^H04", label) else False for label in full_df['ipcrType'].tolist()]
#df = full_df[selector]
#df.reset_index(inplace=True,drop=True)

### Building the citation graph

In [10]:
def citation_adjacency_matrix(df):
    nnz = len(df)

    nodes = []
    for i in range(nnz):
        nodes.append(df.ix[i]['publishedPatentDocNumber'])
    nodeset = set(nodes)

    node2idx = {}
    for idx, node in enumerate(nodes): 
        node2idx[node] = idx

    all_nodes = []
    for i in range(nnz):
        source = df.ix[i]['publishedPatentDocNumber']
        targets = list(set(df.ix[i]['applicantCitations']).union(set(df.ix[i]['examinerCitations'])))
        all_nodes.append(source)
        all_nodes.extend(targets)
    all_nodeset = set(all_nodes)
    all_nodes = list(all_nodeset)

    all_node2idx = {}
    for idx, node in enumerate(nodes):
        all_node2idx[node] = idx

    shift = 0
    for node in all_nodes:
        try:
            dummy = all_node2idx[node]
        except:
            all_node2idx[node] = shift + nnz
            shift += 1

    all_pairs = []
    for i in range(nnz):
        source = df.ix[i]['publishedPatentDocNumber']
        source_idx = all_node2idx[source]
        targets = list(set(df.ix[i]['applicantCitations']).union(set(df.ix[i]['examinerCitations'])))
        for target in targets:
            all_pairs.append([source_idx, all_node2idx[target]])   

    row, col = zip(*all_pairs)
    val = np.ones(len(row)).astype(np.float64) 
    height = max(max(row), max(col)) + 1
    width = height

    A_coo = sparse.coo_matrix((val, (row, col)), shape = (height, width))

    A = sparse.csr_matrix(A_coo)
    A = A + A.T

    B = A[:nnz, :]

    C = np.dot(B, B.T)

    D = A[:nnz, :nnz]

    link_matrix = C + D
    return link_matrix

### Running...

In [11]:
link_matrix = citation_adjacency_matrix(full_df)